### Install and import libraries (Python and R)

In [2]:
# Install libraries not included to Databricks Runtime - already set up to be installed on cluster automatically
#dbutils.library.installPyPI("folium")
#dbutils.library.installPyPI("xlrd")
#dbutils.library.installPyPI("plotly")

In [3]:
%r
# Install libraries not included to Databricks Runtime - already set up to be installed on cluster automatically
#install.packages("readr")

NULL

In [4]:
# Import libraries
import pandas as pd  
import numpy as np
import folium
import folium.plugins as plugins
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

In [5]:
%r
library (SparkR)

### Explore central Denver for areas with the least traffic and most restaurants

### Our datasets
1. Local businesses including restaurants - using Google API and R.
2. Traffic counts and locations - by Denver Open Data Catalog.

### Preparing and cleansing Local Businesses dataset

In [9]:
# Get local business dataset based on selected zones (POC)
larimer = pd.read_csv('/dbfs/FileStore/tables/larimer_square.csv', encoding='latin-1')
larimer.insert(0, 'zone', 'Larimer')
#larimer.head()

pearl = pd.read_csv('/dbfs/FileStore/tables/pearl.csv', encoding='latin-1')
pearl.insert(0, 'zone', 'Pearl')
#pearl.head()

tennyson = pd.read_csv('/dbfs/FileStore/tables/tennyson.csv', encoding='latin-1')
tennyson.insert(0, 'zone', 'Tennyson')
#tennyson.head()

In [10]:
# Create combined zones dataset and cleanse
zones = pd.concat([larimer, pearl, tennyson], sort=False, ignore_index=True)
zones.rename(columns={'geometry.location.lat': 'latitude', 'geometry.location.lng': 'longitude'}, inplace=True)

# Let's keep 'Denver' as a placeholder for traffic counts
#zones = zones[(zones['business_status'] == 'OPERATIONAL')]

# Drop unneeded columns
zones = zones.drop(['Unnamed: 0', 'icon', 'id', 'place_id', 'reference', 'scope', 'plus_code.compound_code', 'plus_code.global_code', 'geometry.viewport.northeast.lat', 'geometry.viewport.northeast.lng', 'geometry.viewport.southwest.lat', 'geometry.viewport.southwest.lng', 'business_status'], axis=1)

# Data conversion
zones['name'] = zones['name'].astype(str)

# Let's make some assumtions and fill in blank price level with average (2)
zones['price_level'] = zones['price_level'].fillna(2)

#zones = zones[(zones['zone'] == input_location_name)]

#zones.head()

In [11]:
# We need to create 'type' feature

# Get rid of future duplicates when business belongs to more than one business type (only for selected types)
zones['food'] = np.where(zones['restaurant'] > 0, 0, zones['food'])
zones['bar'] = np.where(np.logical_or.reduce((zones['restaurant'] > 0, zones['food'] > 0)), 0, zones['bar'])
zones['home_goods_store'] = np.where(np.logical_or.reduce((zones['restaurant'] > 0, zones['food'] > 0, zones['bar'] > 0)), 0, zones['home_goods_store'])
zones['store'] =  np.where(np.logical_or.reduce((zones['restaurant'] > 0, zones['food'] > 0, zones['bar'] > 0, zones['home_goods_store'] > 0)), 0, zones['store'])
zones['clothing_store'] = np.where(np.logical_or.reduce((zones['restaurant'] > 0, zones['food'] > 0, zones['bar'] > 0, zones['home_goods_store'] > 0, zones['store'] > 0)), 0, zones['clothing_store'])
zones['shoe_store'] = np.where(np.logical_or.reduce((zones['restaurant'] > 0, zones['food'] > 0, zones['bar'] > 0, zones['home_goods_store'] > 0, zones['store'] > 0, zones['clothing_store'] > 0)), 0, zones['shoe_store'])

# Unpivot to create type of business
type_filter = ['restaurant', 'food', 'bar', 'home_goods_store', 'store', 'clothing_store', 'shoe_store', 'locality']
id_filter = ['zone', 'name', 'price_level', 'rating', 'latitude', 'longitude']
zones = pd.melt(zones, id_vars=id_filter, value_vars=type_filter, var_name='type')
zones = zones[(zones['value'] == 1)]

#zones.head()

In [12]:
# Split zones dataset into restaurants and other businesses to help displaying on the map
restaurants = zones[(zones['type'] == 'restaurant') | (zones['type'] == 'bar') | (zones['type'] == 'food')]
#restaurants.head()

business = zones[(zones['type'] != 'restaurant') & (zones['type'] != 'bar') & (zones['type'] != 'food')]
#business.head()

In [13]:
displayHTML("""<font size="4" color="blue" face="sans-serif">Street Closure Candidate</font>""")

Street Closure Candidate

In [14]:
# This is only for POC - selected areas only

# Output Variables
zones_select = pd.DataFrame({'zone':['Larimer', 'Pearl', 'Tennyson'],
                   'coordinates':[[39.7482, -104.9986], [39.6893, -104.9804], [39.7711, -105.0440]]})
  

# Convert to Spark dataframe/table
zones_select_spark = spark.createDataFrame(zones_select)
zones_select_spark.write.mode("overwrite").saveAsTable("zones_select")

#display(zones_select)

# Selection lists for widgets
zone_list = spark.sql("select distinct(zone) from zones_select").rdd.map(lambda row : row[0]).collect()
#zone_list.sort()
zone_coordinates = spark.sql("select distinct(coordinates) from zones_select").rdd.map(lambda row : row[0]).collect()

# Create widgets
dbutils.widgets.dropdown("Select Zone", "Pearl", [str(x) for x in zone_list])
#dbutils.widgets.dropdown("Select Coordinates", "[39.6893, -104.9804]", [str(x) for x in zone_coordinates])
#dbutils.widgets.removeAll()

z=zones.drop_duplicates(['zone'])[['zone']]
display(z)

zone
Larimer
Pearl
Tennyson


In [15]:
# Choose area to close (POC - 3 areas)
input_location_name = "Pearl"
#dbutils.widgets.get("Select Zone") 
input_location = [39.6893, -104.9804]
# larimer: [39.7482, -104.9986]
# pearl: [39.6893, -104.9804]
# tennyson: [39.7711, -105.0440]
input_radius = 200
input_radius_str = str(input_radius) + ' meters'

displayHTML("""<font size="8" color="green" face="sans-serif">Pearl</font>""")

Pearl

In [16]:
# This is only for POC - build analysis for one area in a time
onezone = zones[(zones['zone'] == input_location_name)]
onezone.head(10)

,zone,name,price_level,rating,latitude,longitude,type,value
61,Pearl,Sushi Den,3.0,4.7,39.689567,-104.980604,restaurant,1.0
63,Pearl,Izakaya Den,3.0,4.7,39.689680,-104.980587,restaurant,1.0
64,Pearl,Kaos Pizzeria,2.0,4.5,39.690497,-104.980647,restaurant,1.0
65,Pearl,Hanson's Grill & Tavern,2.0,4.0,39.692829,-104.980607,restaurant,1.0
67,Pearl,Sexy Pizza,1.0,4.3,39.687916,-104.980651,restaurant,1.0
69,Pearl,OTOTO,3.0,4.6,39.689265,-104.980706,restaurant,1.0
81,Pearl,Craft Alley,2.0,4.7,39.690286,-104.980665,restaurant,1.0
85,Pearl,Tavern Platt Park,2.0,4.0,39.689847,-104.980641,restaurant,1.0
90,Pearl,Que Bueno Suerte,3.0,4.2,39.689078,-104.980120,restaurant,1.0
94,Pearl,Bird,2.0,4.2,39.688696,-104.980621,restaurant,1.0


### Preparing and cleansing Traffic dataset

In [18]:
# Create traffic dataset
traffic = pd.read_csv('/dbfs/FileStore/tables/geocoded_traffic.csv', encoding='latin-1')
#traffic.head()

In [19]:
# Rename columns
traffic.rename(columns={'geometry.location.lat': 'latitude', 'geometry.location.lng': 'longitude'}, inplace=True)

# Convert dates to datetime
import datetime as dt
traffic['Date'] = pd.to_datetime(traffic['Date'], format = '%Y/%m/%d')

# Drop unneeded columns
traffic = traffic.drop('Unnamed: 0', axis=1)
#traffic = traffic[traffic['Date'].dt.year == 2015].drop('Unnamed: 0', axis=1)

# Add Daily Traffic feature
traffic['daily_traffic'] = traffic['AM_PHV'] + traffic['PM_PHV']

#traffic.head()

In [20]:
# We will look at average/mean daily counts
traffic_grouped = traffic.groupby(["formatted_address", "latitude", "longitude"]).mean().reset_index()

traffic_grouped.head(10)

,formatted_address,latitude,longitude,AM_PHV,PM_PHV,ADT,AAWDT,AAWET,daily_traffic
0,"1 S Monaco Pkwy, Denver, CO 80224, USA",39.716210,-104.912869,361.0,494.0,NaN,NaN,NaN,855.0
1,"10 S Emerson St, Denver, CO 80209, USA",39.716272,-104.976608,208.0,174.0,NaN,2102.0,NaN,382.0
2,"1001 16th St Mall, Denver, CO 80265, USA",39.747799,-104.994906,393.0,821.0,NaN,6647.0,NaN,1214.0
3,"1010 E 13th Ave, Denver, CO 80218, USA",39.736528,-104.974959,890.0,872.0,NaN,12355.0,NaN,1762.0
4,"1055 Logan St, Denver, CO 80203, USA",39.733012,-104.982736,106.0,191.0,NaN,1878.0,NaN,297.0
5,"11194 E 45th Ave, Denver, CO 80239, USA",39.776897,-104.856364,118.0,144.0,NaN,NaN,NaN,262.0
6,"1123 20th St, Denver, CO 80202, USA",39.752071,-104.991186,467.0,335.0,NaN,NaN,NaN,802.0
7,"11855 E 40th Ave, Denver, CO 80239, USA",39.773312,-104.850798,162.0,140.0,NaN,2141.0,NaN,302.0
8,"1200 E 3rd Ave, Denver, CO 80218, USA",39.719842,-104.972541,135.0,256.0,NaN,2547.0,NaN,391.0
9,"1201 S Steele St, Denver, CO 80210, USA",39.694340,-104.950370,199.0,268.0,NaN,2713.0,NaN,467.0


In [21]:
# This is only for POC - build traffic dataset for selected areas only

# Output Variables
traffic_counts = pd.DataFrame({'zone':['Larimer', 'Pearl', 'Tennyson'],
                   'traffic_counts':[13503, 2008, 3738]})
  
# print dataframe
display(traffic_counts)


zone,traffic_counts
Larimer,13503
Pearl,2008
Tennyson,3738


### Conversion to Spark dataframes and Databricks tables
We have a mixed team with Python, R and SQL skills, this helps us to share and reuse code created with different languages

In [23]:
# Convert pandas dataframes into Spark dataframe and then save as tables to share beetween Python and R code
zones_spark = spark.createDataFrame(zones)
zones_spark.write.mode("overwrite").saveAsTable("zones")

onezone_spark = spark.createDataFrame(onezone)
onezone_spark.write.mode("overwrite").saveAsTable("onezone")

restaurants_spark = spark.createDataFrame(restaurants)
restaurants_spark.write.mode("overwrite").saveAsTable("restaurants")

business_spark = spark.createDataFrame(business)
business_spark.write.mode("overwrite").saveAsTable("business")

traffic_spark = spark.createDataFrame(traffic_grouped)
traffic_spark.write.mode("overwrite").saveAsTable("traffic")

traffic_counts_spark = spark.createDataFrame(traffic_counts)
traffic_counts_spark.write.mode("overwrite").saveAsTable("trafficcounts")

### Build closure area map with Local Businesses and Traffic

In [25]:
# Get traffic locations to plot on the map
traffic_locations = traffic_grouped[['latitude', 'longitude']]
traffic_locationlist = traffic_locations.values.tolist()
#len(traffic_locationlist)

# Get restaurants locations to plot on the map
restaurants_locations = restaurants[['latitude', 'longitude']]
restaurants_locationlist = restaurants_locations.values.tolist()
#len(restaurants_locationlist)

# Get businesses locations to plot on the map
business_locations = business[['latitude', 'longitude']]
business_locationlist = business_locations.values.tolist()
#len(business_locationlist)

# Get all local businesses locations to plot on the map
zones_locations = zones[['latitude', 'longitude']]
zones_locationlist = zones_locations.values.tolist()
#len(zones_locationlist)

In [26]:
# THIS IS FINAL MAP

# Build Denver map
map_denver = folium.Map(
      location = input_location,
      tiles = 'CartoDB positron',
      zoom_start = 22)

# Show coordinates to help choose a location
map_denver.add_child(folium.LatLngPopup())

# Add closure Marker
folium.Marker(
      location = input_location,
      popup = 'Closure Area',
      icon = folium.Icon(color='green', icon='times-circle', prefix='fa')
      ).add_to(map_denver)

# Circle closure area
folium.Circle(
      location = input_location,
      popup = 'Closure Area',
      radius = input_radius,
      color = 'green',
      #fill = True,
      #fill_color = 'green'
   ).add_to(map_denver)
    
# Restaurants are Markers
for point in range(0, len(restaurants_locationlist)):
  folium.Marker(
      restaurants_locationlist[point],
      #popup = restaurants['type'][point],
      #popup=folium.Popup(restaurants['name'][point], parse_html=True),
      #tooltip = restaurants['price_level'][point],
      icon = folium.Icon(color='blue', icon='cutlery', prefix='fa')
      ).add_to(map_denver)

# Local Businesses are Markers
for point in range(0, len(business_locationlist)):
  folium.Marker(
      business_locationlist[point],
      #tooltip = business['name'][point],
      icon = folium.Icon(color='orange', icon='briefcase', prefix='fa')
      ).add_to(map_denver)
  
# Traffic is Car Marker or Heatmap?
for point in range(0, len(traffic_locationlist)):
  folium.Marker(
      traffic_locationlist[point],
      tooltip = traffic['daily_traffic'][point],
      icon = folium.Icon(color='red', icon='car', prefix='fa')
      ).add_to(map_denver)
    
#traffic_array = traffic_grouped[['latitude', 'longitude']].to_numpy()
#HeatMap(
#     traffic_array,
#     radius=100,
#     max_zoom = 17
#      ).add_to(map_denver)

# Show results
map_denver

Make this Notebook Trusted to load map: File -> Trust Notebook

In [27]:
displayHTML("""<font size="10" color="orange" face="sans-serif">Cost Benefit Analysis of Street Closure ($ daily)</font>""")

Cost Benefit Analysis of Street Closure ($ daily)

In [28]:
# We created costs and benefits reference dataset
benefits = pd.read_excel('/dbfs/FileStore/tables/Benefit_reference.xlsx', encoding='latin-1')
benefits.head(20)

,Type,price_level,increased_hourly_capacity,hours_open,dollars_per_meal,dollars_per_purchase,profit_margin,likelihood_to_buy,Increase_in_staff_hours,staff_per_hour,salary_per_hour,tip_per_hour,delivery_flag,take_out_dollars,loss_take_out,car_impact,drive_min,drive_value_min
0,restaurant,1,20,10,7,0,0.25,1.00,12.0,2,15,5,0,0,0.00,0,0,0.00
1,restaurant,2,20,10,10,0,0.25,1.00,12.0,2,15,5,0,0,0.00,0,0,0.00
2,restaurant,3,20,10,25,0,0.25,1.00,12.0,2,15,10,0,0,0.00,0,0,0.00
3,restaurant,4,20,10,40,0,0.25,1.00,12.0,2,30,20,0,0,0.00,0,0,0.00
4,food,1,20,10,5,0,0.25,1.00,12.0,2,15,5,1,15,0.25,0,0,0.00
5,food,2,20,10,10,0,0.25,1.00,12.0,2,15,5,1,15,0.25,0,0,0.00
6,food,3,20,10,25,0,0.25,1.00,12.0,2,15,10,1,15,0.25,0,0,0.00
7,food,4,20,10,40,0,0.25,1.00,12.0,2,30,20,1,15,0.25,0,0,0.00
8,home_goods_store,0,20,8,0,75,0.60,0.25,9.6,2,20,0,0,0,0.00,0,0,0.00
9,store,0,20,8,0,50,0.60,0.25,9.6,2,20,0,0,0,0.00,0,0,0.00


In [29]:
# Convert into Spark DataFrame
benefits_spark = spark.createDataFrame(benefits)
benefits_spark.write.mode("overwrite").saveAsTable("benefits")

In [30]:
benefits_spark.write.format("delta").mode("overwrite").save("/delta/benefits/")

In [31]:
%sql 
DROP TABLE IF EXISTS benefits_delta;

CREATE TABLE benefits_delta
USING delta
LOCATION '/delta/benefits/';

In [32]:
%sql
DESCRIBE HISTORY benefits_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics
13,2020-06-11T14:58:33.000+0000,6132610664165234,yuliya.shvets@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4032658702940706),0611-143020-cash683,12,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 33700, numOutputRows -> 17, numParts -> 0)"
12,2020-06-11T14:47:19.000+0000,6132610664165234,yuliya.shvets@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4032658702940706),0611-143020-cash683,11,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 33700, numOutputRows -> 17, numParts -> 0)"
11,2020-06-11T14:41:46.000+0000,6132610664165234,yuliya.shvets@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4032658702940706),0611-143020-cash683,10,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 33700, numOutputRows -> 17, numParts -> 0)"
10,2020-06-10T23:14:11.000+0000,6132610664165234,yuliya.shvets@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4032658702940706),0610-204827-chile236,9,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 33700, numOutputRows -> 17, numParts -> 0)"
9,2020-06-10T23:11:42.000+0000,6132610664165234,yuliya.shvets@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4032658702940706),0610-204827-chile236,8,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 33700, numOutputRows -> 17, numParts -> 0)"
8,2020-06-10T20:57:32.000+0000,6132610664165234,yuliya.shvets@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4032658702940706),0610-204827-chile236,7,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 33700, numOutputRows -> 17, numParts -> 0)"
7,2020-06-09T23:56:01.000+0000,6132610664165234,yuliya.shvets@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4032658702940706),0609-224041-acme589,6,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 33700, numOutputRows -> 17, numParts -> 0)"
6,2020-06-09T23:33:04.000+0000,6132610664165234,yuliya.shvets@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4032658702940706),0609-224041-acme589,5,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 33700, numOutputRows -> 17, numParts -> 0)"
5,2020-06-09T23:23:28.000+0000,6132610664165234,yuliya.shvets@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4032658702940706),0609-224041-acme589,4,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 33700, numOutputRows -> 17, numParts -> 0)"
4,2020-06-09T19:07:45.000+0000,6132610664165234,yuliya.shvets@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4032658702940706),0609-163649-buys419,3,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 33700, numOutputRows -> 17, numParts -> 0)"


In [33]:
%sql
-- See a particular version
SELECT car_impact, drive_min, drive_value_min
FROM benefits_delta VERSION AS OF 0
WHERE Type = 'traffic' 
ORDER BY price_level ASC;

car_impact,drive_min,drive_value_min
1,10,0.36


In [34]:
%sql
DROP TABLE IF EXISTS StreetOp;

In [35]:
%sql
CREATE TABLE StreetOp as 
SELECT z.zone, z.name, z.price_level, z.rating, z.latitude, z.longitude, CASE WHEN z.type = 'locality' THEN 'traffic' ELSE z.type END as type, z.value, b.increased_hourly_capacity, b.hours_open, b.dollars_per_meal, b.dollars_per_purchase, b.profit_margin, b.likelihood_to_buy, b.Increase_in_staff_hours, b.staff_per_hour, b.salary_per_hour, b.tip_per_hour, b.delivery_flag, b.take_out_dollars, b.loss_take_out, b.car_impact, b.drive_min, b.drive_value_min, coalesce (t.traffic_counts,0) as traffic_counts
FROM onezone z
LEFT OUTER JOIN benefits_delta VERSION AS OF 0 b ON CASE WHEN z.type = 'locality' THEN 'traffic' ELSE z.type END = b.Type AND CASE WHEN z.type in ('restaurant', 'bar', 'food') THEN z.price_level = b.price_level ELSE z.price_level = z.price_level END 
LEFT OUTER JOIN trafficcounts t ON z.zone = t.zone AND z.type = 'locality';

SELECT * FROM StreetOp;

zone,name,price_level,rating,latitude,longitude,type,value,increased_hourly_capacity,hours_open,dollars_per_meal,dollars_per_purchase,profit_margin,likelihood_to_buy,Increase_in_staff_hours,staff_per_hour,salary_per_hour,tip_per_hour,delivery_flag,take_out_dollars,loss_take_out,car_impact,drive_min,drive_value_min,traffic_counts
Pearl,"YAO Herbal Apothecary, Clinic & Tea Market",2.0,4.8,39.692836,-104.9795,food,1.0,20,10,10,0,0.25,1.0,12.0,2,15,5,1,15,0.25,0,0,0.0,0
Pearl,Second Star to the Right Children's Books,2.0,4.9,39.6885179,-104.98062009999998,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0
Pearl,Denver Doll Emporium Box,2.0,4.0,39.6880805,-104.98018789999999,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0
Pearl,Old South Frame & Gallery,2.0,5.0,39.687816600000005,-104.9803187,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0
Pearl,Common Threads Denver,2.0,4.4,39.687994200000006,-104.98059409999999,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0
Pearl,Denver,2.0,null,39.739235799999996,-104.990251,traffic,1.0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,1,10,0.36,2008
Pearl,Craft Alley,2.0,4.7,39.690286,-104.98066499999999,restaurant,1.0,20,10,10,0,0.25,1.0,12.0,2,15,5,0,0,0.0,0,0,0.0,0
Pearl,Tavern Platt Park,2.0,4.0,39.6898469,-104.98064099999999,restaurant,1.0,20,10,10,0,0.25,1.0,12.0,2,15,5,0,0,0.0,0,0,0.0,0
Pearl,Que Bueno Suerte,3.0,4.2,39.6890777,-104.98011969999999,restaurant,1.0,20,10,25,0,0.25,1.0,12.0,2,15,10,0,0,0.0,0,0,0.0,0
Pearl,Sushi Den,3.0,4.7,39.68956729999999,-104.980604,restaurant,1.0,20,10,25,0,0.25,1.0,12.0,2,15,10,0,0,0.0,0,0,0.0,0


In [36]:
%r
StreetOp <- sql("SELECT * FROM StreetOp")

display(StreetOp)

zone,name,price_level,rating,latitude,longitude,type,value,increased_hourly_capacity,hours_open,dollars_per_meal,dollars_per_purchase,profit_margin,likelihood_to_buy,Increase_in_staff_hours,staff_per_hour,salary_per_hour,tip_per_hour,delivery_flag,take_out_dollars,loss_take_out,car_impact,drive_min,drive_value_min,traffic_counts
Pearl,"YAO Herbal Apothecary, Clinic & Tea Market",2.0,4.8,39.692836,-104.9795,food,1.0,20,10,10,0,0.25,1.0,12.0,2,15,5,1,15,0.25,0,0,0.0,0
Pearl,Second Star to the Right Children's Books,2.0,4.9,39.6885179,-104.98062009999998,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0
Pearl,Denver Doll Emporium Box,2.0,4.0,39.6880805,-104.98018789999999,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0
Pearl,Old South Frame & Gallery,2.0,5.0,39.687816600000005,-104.9803187,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0
Pearl,Common Threads Denver,2.0,4.4,39.687994200000006,-104.98059409999999,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0
Pearl,Denver,2.0,null,39.739235799999996,-104.990251,traffic,1.0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,1,10,0.36,2008
Pearl,Craft Alley,2.0,4.7,39.690286,-104.98066499999999,restaurant,1.0,20,10,10,0,0.25,1.0,12.0,2,15,5,0,0,0.0,0,0,0.0,0
Pearl,Tavern Platt Park,2.0,4.0,39.6898469,-104.98064099999999,restaurant,1.0,20,10,10,0,0.25,1.0,12.0,2,15,5,0,0,0.0,0,0,0.0,0
Pearl,Que Bueno Suerte,3.0,4.2,39.6890777,-104.98011969999999,restaurant,1.0,20,10,25,0,0.25,1.0,12.0,2,15,10,0,0,0.0,0,0,0.0,0
Pearl,Sushi Den,3.0,4.7,39.68956729999999,-104.980604,restaurant,1.0,20,10,25,0,0.25,1.0,12.0,2,15,10,0,0,0.0,0,0,0.0,0


In [37]:
%r
#Benefits
StreetOp$RestBenefitsRev <- (StreetOp$increased_hourly_capacity) * (StreetOp$hours_open) * (StreetOp$dollars_per_meal) * (StreetOp$profit_margin)

StreetOp$RestBenefitsJobs <- (StreetOp$Increase_in_staff_hours) * (StreetOp$staff_per_hour) * (StreetOp$salary_per_hour) * (StreetOp$tip_per_hour)

StreetOp$RetBenefitsSales <- (StreetOp$increased_hourly_capacity) * (StreetOp$likelihood_to_buy) * (StreetOp$dollars_per_purchase) * (StreetOp$profit_margin)

StreetOp$RetBenefitsJobs <- (StreetOp$Increase_in_staff_hours) * (StreetOp$staff_per_hour) * (StreetOp$salary_per_hour)

StreetOp$BusBenefitsRev <- StreetOp$RestBenefitsRev + StreetOp$RetBenefitsSales

StreetOp$BusBenefitsJobs <- StreetOp$RestBenefitsJobs + StreetOp$RetBenefitsJobs

#Costs

StreetOp$CostGov <- (StreetOp$hours_open) * (StreetOp$salary_per_hour) * (-1)

StreetOp$CostRestaurants <- (StreetOp$take_out_dollars) * (StreetOp$delivery_flag) * (StreetOp$loss_take_out) * (-1)

StreetOp$CostTraffic <- (StreetOp$traffic_counts)  * (StreetOp$drive_min) * (StreetOp$drive_value_min) * (-1)

In [38]:
%r
#Benefits
StreetOp$TotalBenefits <- StreetOp$RestBenefitsRev + StreetOp$RestBenefitsJobs + StreetOp$RetBenefitsSales + StreetOp$RetBenefitsJobs

#Cost
StreetOp$TotalCost <- StreetOp$CostGov + StreetOp$CostRestaurants + StreetOp$CostTraffic

#Net
StreetOp$Net <- (StreetOp$TotalBenefits+StreetOp$TotalCost) 

#Ratio
StreetOp$CostBenefitsRatio <- (StreetOp$TotalCost/StreetOp$TotalBenefits)*-1

In [39]:
%r
createOrReplaceTempView(StreetOp, "StreetOp")
display (StreetOp)

zone,name,price_level,rating,latitude,longitude,type,value,increased_hourly_capacity,hours_open,dollars_per_meal,dollars_per_purchase,profit_margin,likelihood_to_buy,Increase_in_staff_hours,staff_per_hour,salary_per_hour,tip_per_hour,delivery_flag,take_out_dollars,loss_take_out,car_impact,drive_min,drive_value_min,traffic_counts,RestBenefitsRev,RestBenefitsJobs,RetBenefitsSales,RetBenefitsJobs,BusBenefitsRev,BusBenefitsJobs,CostGov,CostRestaurants,CostTraffic,TotalBenefits,TotalCost,Net,CostBenefitsRatio
Pearl,"YAO Herbal Apothecary, Clinic & Tea Market",2.0,4.8,39.692836,-104.9795,food,1.0,20,10,10,0,0.25,1.0,12.0,2,15,5,1,15,0.25,0,0,0.0,0,500.0,1800.0,0.0,360.0,500.0,2160.0,-150.0,-3.75,-0.0,2660.0,-153.75,2506.25,0.05780075187969925
Pearl,Second Star to the Right Children's Books,2.0,4.9,39.6885179,-104.98062009999998,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0,0.0,0.0,150.0,384.0,150.0,384.0,-160.0,-0.0,-0.0,534.0,-160.0,374.0,0.299625468164794
Pearl,Denver Doll Emporium Box,2.0,4.0,39.6880805,-104.98018789999999,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0,0.0,0.0,150.0,384.0,150.0,384.0,-160.0,-0.0,-0.0,534.0,-160.0,374.0,0.299625468164794
Pearl,Old South Frame & Gallery,2.0,5.0,39.687816600000005,-104.9803187,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0,0.0,0.0,150.0,384.0,150.0,384.0,-160.0,-0.0,-0.0,534.0,-160.0,374.0,0.299625468164794
Pearl,Common Threads Denver,2.0,4.4,39.687994200000006,-104.98059409999999,store,1.0,20,8,0,50,0.6,0.25,9.6,2,20,0,0,0,0.0,0,0,0.0,0,0.0,0.0,150.0,384.0,150.0,384.0,-160.0,-0.0,-0.0,534.0,-160.0,374.0,0.299625468164794
Pearl,Denver,2.0,null,39.739235799999996,-104.990251,traffic,1.0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,1,10,0.36,2008,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-7228.8,0.0,-7228.8,-7228.8,null
Pearl,Craft Alley,2.0,4.7,39.690286,-104.98066499999999,restaurant,1.0,20,10,10,0,0.25,1.0,12.0,2,15,5,0,0,0.0,0,0,0.0,0,500.0,1800.0,0.0,360.0,500.0,2160.0,-150.0,-0.0,-0.0,2660.0,-150.0,2510.0,0.05639097744360902
Pearl,Tavern Platt Park,2.0,4.0,39.6898469,-104.98064099999999,restaurant,1.0,20,10,10,0,0.25,1.0,12.0,2,15,5,0,0,0.0,0,0,0.0,0,500.0,1800.0,0.0,360.0,500.0,2160.0,-150.0,-0.0,-0.0,2660.0,-150.0,2510.0,0.05639097744360902
Pearl,Que Bueno Suerte,3.0,4.2,39.6890777,-104.98011969999999,restaurant,1.0,20,10,25,0,0.25,1.0,12.0,2,15,10,0,0,0.0,0,0,0.0,0,1250.0,3600.0,0.0,360.0,1250.0,3960.0,-150.0,-0.0,-0.0,5210.0,-150.0,5060.0,0.028790786948176585
Pearl,Sushi Den,3.0,4.7,39.68956729999999,-104.980604,restaurant,1.0,20,10,25,0,0.25,1.0,12.0,2,15,10,0,0,0.0,0,0,0.0,0,1250.0,3600.0,0.0,360.0,1250.0,3960.0,-150.0,-0.0,-0.0,5210.0,-150.0,5060.0,0.028790786948176585


### Prepare visuals for final presentation / dashboard using Databricks charting options and external libraries

In [41]:
%r
StreetOpFinal <- sql("SELECT distinct type as `Business Type`, price_level as `Price level`, sum (BusBenefitsRev) as `Business Benefit`, sum (BusBenefitsJobs) as `Jobs Benefit`, sum (TotalBenefits) as `Benefit`, sum (CostGov) as `Government Cost`, sum (CostRestaurants) as `Restaurant Cost`, sum (TotalCost) as `Cost`, sum (Net) as `Net Benefit` FROM StreetOp GROUP BY type, price_level")
display(StreetOpFinal)

Business Type,Price level,Business Benefit,Jobs Benefit,Benefit,Government Cost,Restaurant Cost,Cost,Net Benefit
food,1.0,250.0,2160.0,2410.0,-150.0,-3.75,-153.75,2256.25
food,2.0,1000.0,4320.0,5320.0,-300.0,-7.5,-307.5,5012.5
restaurant,2.0,2500.0,10800.0,13300.0,-750.0,0.0,-750.0,12550.0
restaurant,3.0,5000.0,15840.0,20840.0,-600.0,0.0,-600.0,20240.0
traffic,2.0,0.0,0.0,0.0,0.0,0.0,-7228.8,-7228.8
store,2.0,600.0,1536.0,2136.0,-640.0,0.0,-640.0,1496.0
restaurant,1.0,350.0,2160.0,2510.0,-150.0,0.0,-150.0,2360.0


In [42]:
%r
StreetOpBusType <- sql("SELECT distinct type as `Business Type`, sum (BusBenefitsRev) as `Business Benefit`, sum (BusBenefitsJobs) as `Jobs Benefit`, sum (TotalBenefits) as `Benefit`, sum (CostGov) as `Government Cost`, sum (CostRestaurants) as `Restaurant Cost`, sum (TotalCost) as `Cost`, sum (Net) as `Net Benefit` FROM StreetOp WHERE type != 'traffic' GROUP BY type")
display(StreetOpBusType)

Business Type,Business Benefit,Jobs Benefit,Benefit,Government Cost,Restaurant Cost,Cost,Net Benefit
food,1250.0,6480.0,7730.0,-450.0,-11.25,-461.25,7268.75
store,600.0,1536.0,2136.0,-640.0,0.0,-640.0,1496.0
restaurant,7850.0,28800.0,36650.0,-1500.0,0.0,-1500.0,35150.0


In [43]:
%r
StreetOpTraffic <- sql("SELECT sum (CostTraffic) as `Traffic Cost` FROM StreetOp WHERE type = 'traffic'")
display(StreetOpTraffic)

Traffic Cost
-7228.8


In [44]:
displayHTML("""<font size="4" color="blue" face="sans-serif">Net Benefit by Business Type</font>""")

Net Benefit by Business Type

In [45]:
displayHTML("""<font size="4" color="black" face="sans-serif">Businesses that would benefit the most from the street closure</font>""")

Businesses that would benefit the most from the street closure

In [46]:
%r
display(StreetOpBusType)

Business Type,Business Benefit,Jobs Benefit,Benefit,Government Cost,Restaurant Cost,Cost,Net Benefit
food,1250.0,6480.0,7730.0,-450.0,-11.25,-461.25,7268.75
store,600.0,1536.0,2136.0,-640.0,0.0,-640.0,1496.0
restaurant,7850.0,28800.0,36650.0,-1500.0,0.0,-1500.0,35150.0


In [47]:
%r
display(StreetOpFinal)

Business Type,Price level,Business Benefit,Jobs Benefit,Benefit,Government Cost,Restaurant Cost,Cost,Net Benefit
food,1.0,250.0,2160.0,2410.0,-150.0,-3.75,-153.75,2256.25
food,2.0,1000.0,4320.0,5320.0,-300.0,-7.5,-307.5,5012.5
restaurant,2.0,2500.0,10800.0,13300.0,-750.0,0.0,-750.0,12550.0
restaurant,3.0,5000.0,15840.0,20840.0,-600.0,0.0,-600.0,20240.0
traffic,2.0,0.0,0.0,0.0,0.0,0.0,-7228.8,-7228.8
store,2.0,600.0,1536.0,2136.0,-640.0,0.0,-640.0,1496.0
restaurant,1.0,350.0,2160.0,2510.0,-150.0,0.0,-150.0,2360.0


In [48]:
displayHTML("""<font size="4" color="blue" face="sans-serif">Total Benefit</font>""")

Total Benefit

In [49]:
displayHTML("""<font size="4" color="black" face="sans-serif">Increased restaurant revenue and retail sales, vs providing more jobs</font>""")

Increased restaurant revenue and retail sales, vs providing more jobs

In [50]:
%r
display(StreetOpBusType)

Business Type,Business Benefit,Jobs Benefit,Benefit,Government Cost,Restaurant Cost,Cost,Net Benefit
food,1250.0,6480.0,7730.0,-450.0,-11.25,-461.25,7268.75
store,600.0,1536.0,2136.0,-640.0,0.0,-640.0,1496.0
restaurant,7850.0,28800.0,36650.0,-1500.0,0.0,-1500.0,35150.0


In [51]:
displayHTML("""<font size="4" color="blue" face="sans-serif">Total Cost</font>""")

Total Cost

In [52]:
displayHTML("""<font size="4" color="black" face="sans-serif">City cost for road closures, food pick-up impact, and traffic impact</font>""")

City cost for road closures, food pick-up impact, and traffic impact

In [53]:
%r
display(StreetOpFinal)

Business Type,Price level,Business Benefit,Jobs Benefit,Benefit,Government Cost,Restaurant Cost,Cost,Net Benefit
food,1.0,250.0,2160.0,2410.0,-150.0,-3.75,-153.75,2256.25
food,2.0,1000.0,4320.0,5320.0,-300.0,-7.5,-307.5,5012.5
restaurant,2.0,2500.0,10800.0,13300.0,-750.0,0.0,-750.0,12550.0
restaurant,3.0,5000.0,15840.0,20840.0,-600.0,0.0,-600.0,20240.0
traffic,2.0,0.0,0.0,0.0,0.0,0.0,-7228.8,-7228.8
store,2.0,600.0,1536.0,2136.0,-640.0,0.0,-640.0,1496.0
restaurant,1.0,350.0,2160.0,2510.0,-150.0,0.0,-150.0,2360.0


In [54]:
%r
display(StreetOpTraffic)

Traffic Cost
-7228.8


In [55]:
# Convert back to Spark and pandas to build some visuals using Python
StreetOpFinal = spark.table("StreetOp")
StreetOpFinal = StreetOpFinal.select("*").toPandas()
#StreetOpFinal.head()

benefit = StreetOpFinal["TotalBenefits"].sum()
cost = StreetOpFinal["TotalCost"].sum()
net = StreetOpFinal["Net"].sum()

In [56]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Indicator(
    mode = "number",
    value = benefit,
    number = {"prefix": "$", "valueformat": ",f."},
    name = "Benefit",
    title = {"text": "Total Benefit<br><span style='font-size:10;color:green'>"},
    domain = {'row': 0, 'column': 1}))

In [57]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = cost,
    number = {"prefix": "$", "valueformat": ",f."},
    name = "Cost", 
    title = {"text": "Total Cost<br><span style='font-size:10;color:green'>"},
    domain = {'row': 0, 'column': 1}))

In [58]:
displayHTML("""<font size="4" color="blue" face="sans-serif">Net Benefit</font>""")

Net Benefit

In [59]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = net,
    number = {"prefix": "$", "valueformat": ",f."},
    name = "Net Benefit",
    title = {"text": "Net<br><span style='font-size:10;color:green'>"},
    domain = {'row': 0, 'column': 1}))